In [1]:
# ------------------------------------------------------------------------------ #
# @Author:        F. Paul Spitzner
# @Email:         paul.spitzner@ds.mpg.de
# @Created:       2023-08-04 11:59:06
# @Last Modified: 2023-08-04 11:59:15
# ------------------------------------------------------------------------------ #
# Run on the cluster, using dask.
# Analyses all units and saves a large dataframe with everything
# that is needed.
# ------------------------------------------------------------------------------ #

%load_ext ipy_dict_hierarchy
%load_ext watermark

import logging
logging.basicConfig(
    format="%(asctime)s | %(levelname)-s | %(name)-s | %(funcName)-s | %(message)s",
    level=logging.WARNING,
)
log = logging.getLogger("notebook")
log.setLevel("DEBUG")

import re
import glob
import h5py
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import dask

from tqdm import tqdm

# also needs to be added for each dask-worker
extra_path = os.path.abspath('../')
sys.path.append(extra_path)
log.info(f"project directory: {extra_path}")

from ana import utility as utl
utl.log.setLevel("ERROR")

output_dir = "/data.nst/share/data/mouse_visual_timescales/data/cache/"
output_name = "all_units_no_merging_block_6_8"
# specify the path as closely as possible, we search recursively through all subdirs
data_dir = "/data.nst/share/data/allen_visual_coding_neuropixels/"

2023-08-31 14:01:03,737 | INFO | notebook | <module> | project directory: /data.nst/share/data/mouse_visual_timescales/experiment_analysis


In [2]:
meta_df = utl.all_unit_metadata(data_dir, reload=False)
meta_df = utl.load_spikes(meta_df, format="numpy")
meta_df = utl.default_filter(meta_df, trim=False)  # only updates the status column
meta_df["spiketimes"] = meta_df.apply(
    lambda row: utl.prepare_spike_times(row["spiketimes"], row["stimulus"]),
    axis=1,
)

meta_df.tail()

Fetching metadata from sessions:  22%|██▏       | 13/58 [00:11<00:31,  1.42it/s]

In [1]:
meta_df

NameError: name 'meta_df' is not defined

In [3]:
# 23293 rows with success
print(len(meta_df))
print(len(meta_df.query("invalid_spiketimes_check == 'SUCCESS'")))
print(len(meta_df.query("invalid_spiketimes_check != 'SUCCESS'")))

31190
30339
851


In [3]:
# to reassemble after anlaysis, we need an non-ambigous index
meta_df.set_index(['unit_id', 'stimulus', 'session', 'block'], inplace=True)
assert meta_df.index.is_unique
meta_df = meta_df.query("invalid_spiketimes_check == 'SUCCESS'")
meta_df = meta_df.query("(block == '8.0') | (block == '6.0')")
meta_df

,,,,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,spiketimes
unit_id,stimulus,session,block,,,,,,,
950987325,natural_movie_one_more_repeats,774875821,8.0,VISam,SUCCESS,900.415039,3.463957,/data.nst/share/data/allen_visual_coding_neuro...,3119,"[0.34570312, 0.38378906, 1.4921875, 1.9301758,..."
950987344,natural_movie_one_more_repeats,774875821,8.0,VISam,SUCCESS,890.576660,1.682056,/data.nst/share/data/allen_visual_coding_neuro...,1498,"[1.09375, 1.3823242, 2.4262695, 2.8881836, 2.9..."
950987340,natural_movie_one_more_repeats,774875821,8.0,VISam,SUCCESS,899.042969,2.200118,/data.nst/share/data/allen_visual_coding_neuro...,1978,"[1.3613281, 1.3852539, 1.7163086, 2.3188477, 2..."
950987352,natural_movie_one_more_repeats,774875821,8.0,VISam,SUCCESS,900.287598,4.218654,/data.nst/share/data/allen_visual_coding_neuro...,3798,"[0.03515625, 0.04736328, 0.056640625, 0.103027..."
950987362,natural_movie_one_more_repeats,774875821,8.0,VISam,SUCCESS,900.726074,6.815613,/data.nst/share/data/allen_visual_coding_neuro...,6139,"[0.015136719, 0.05029297, 0.1015625, 0.1137695..."
...,...,...,...,...,...,...,...,...,...,...
951871993,natural_movie_three,746083955,6.0,VISrl,SUCCESS,600.057373,11.000614,/data.nst/share/data/allen_visual_coding_neuro...,6601,"[0.07299805, 0.26049805, 0.28198242, 0.8112793..."
951872268,natural_movie_three,746083955,6.0,VISrl,SUCCESS,597.371582,4.288787,/data.nst/share/data/allen_visual_coding_neuro...,2562,"[0.75, 1.2885742, 3.0444336, 3.5073242, 3.7246..."
951872311,natural_movie_three,746083955,6.0,VISrl,SUCCESS,588.665527,0.482447,/data.nst/share/data/allen_visual_coding_neuro...,284,"[12.424316, 12.4375, 13.071289, 15.591309, 16...."


In [5]:
meta_df.reset_index().to_feather(f"{output_dir}/{output_name}_test.feather")

In [4]:
hde_settings = {
    "number_of_bootstraps_R_tot": 0,
    "number_of_bootstraps_R_max": 250,
    "timescale_minimum_past_range": 30 / 1000,
    "embedding_number_of_bins_set": [5],
    "estimation_method": "shuffling",
    "persistent_analysis": False,
    # fmt: off
    "embedding_past_range_set": [
        0.005, 0.00561, 0.00629, 0.00706, 0.00792, 0.00889, 0.00998, 0.01119, 0.01256,
        0.01409, 0.01581, 0.01774, 0.01991, 0.02233, 0.02506, 0.02812, 0.03155, 0.0354,
        0.03972, 0.04456, 0.05, 0.0561, 0.06295, 0.06441, 0.06591, 0.06745, 0.06902,
        0.07063, 0.07227, 0.07396, 0.07568, 0.07744, 0.07924, 0.08109, 0.08298, 0.08491,
        0.08689, 0.08891, 0.09099, 0.0931, 0.09527, 0.09749, 0.09976, 0.10209, 0.10446,
        0.1069, 0.10939, 0.11194, 0.11454, 0.11721, 0.11994, 0.12274, 0.12559, 0.12852,
        0.13151, 0.13458, 0.13771, 0.14092, 0.1442, 0.14756, 0.151, 0.15451, 0.15811,
        0.1618, 0.16557, 0.16942, 0.17337, 0.17741, 0.18154, 0.18577, 0.19009, 0.19452,
        0.19905, 0.20369, 0.20843, 0.21329, 0.21826, 0.22334, 0.25059, 0.28117, 0.31548,
        0.35397, 0.39716, 0.44563, 0.5, 0.56101, 0.62946, 0.70627, 0.79245, 0.88914,
        0.99763, 1.11936, 1.25594, 1.40919, 1.58114, 1.77407, 1.99054, 2.23342, 2.50594,
        2.81171, 3.15479, 3.53973, 3.97164, 4.45625, 5.0,
    ],
    # fmt: on
}

mre_settings = {
    "bin_size": 0.005,  # 5 ms
    "tmin": 0.03,
    "tmax": 10.0,
}


def mre_wrapper(data, settings):

    logging.getLogger("mrestimator").setLevel("ERROR")
    import mrestimator as mre
    mre.disable_progressbar()

    data = data.squeeze()
    assert data.ndim == 1, "data must be 1D, this is the simple one-unit wrapper"

    binned_spikes = utl.binned_spike_count(data, bin_size=settings["bin_size"])

    rk = mre.coefficients(
        binned_spikes,
        method="ts",  # method does not matter for single unit
        steps=(
            int(settings["tmin"] / settings["bin_size"]),
            int(settings["tmax"] / settings["bin_size"]),
        ),
        dt=settings["bin_size"],
        dtunit="s",
    )

    fit_single = mre.fit(rk, fitfunc=mre.f_exponential_offset)
    fit_double = mre.fit(rk, fitfunc=mre.f_two_timescales)

    details_single = fit_single._asdict()
    details_double = fit_double._asdict()

    details_single["fitfunc"] = details_single["fitfunc"].__name__
    details_double["fitfunc"] = details_double["fitfunc"].__name__

    for key in settings.keys():
        details_single[key] = settings[key]
        details_double[key] = settings[key]
    
    # steps might get too big and are easy to reconstruct
    details_single.pop("steps", None)
    details_double.pop("steps", None)

    res = {
        "tau_single": fit_single.tau,
        "tau_double": fit_double.tau,
        "details_tau_single": details_single,
        "details_tau_double": details_double,
    }

    return res


def full_analysis(spikes):
    """
    Take one set of spikes, run hdestimator and mrestimator.

    # Parameters
    spikes : 1d numpy array
        flat list of spike times for a single unit. nans are removed.
    """

    logging.getLogger("hdestimator").setLevel("ERROR")
    import hdestimator as hde

    # remove nan-padding
    spikes = spikes.squeeze()
    spikes = spikes[np.isfinite(spikes)]

    try:
        hde_res = hde.api.wrapper(spike_times=spikes, settings=hde_settings)
        for key in ["plot_AIS", "plot_settings", "plot_color", "ANALYSIS_DIR"]:
            hde_res["settings"].pop(key, None)
    except:
        hde_res = dict()
        hde_res["R_tot"] = np.nan
        hde_res["tau_R"] = np.nan

    try:
        mre_res = mre_wrapper(spikes, mre_settings)
    except:
        mre_res = dict()
        mre_res["tau_single"] = np.nan
        mre_res["tau_double"] = np.nan
        mre_res["details_tau_single"] = dict()
        mre_res["details_tau_double"] = dict()

    return hde_res, mre_res


def dask_it(itertuple):
    """
    Idea is to delegate this to a dask worker and get a dictionary back
    that has everything we need. Every worker gets a one-row dataframe

    Retruns a tuple of (index, hde_res, mre_res)
    index is a pandas index so we can re-insert into the outer dataframe,
    the others are dicts with results
    """

    index, iterrow = itertuple

    # dont forget to prepare the spikes beforehand
    spikes = iterrow["spiketimes"].squeeze()
    
    hde_res, mre_res = full_analysis(spikes)

    return index, hde_res, mre_res


In [ ]:
# manually test for a single unit
for itertuple in meta_df.iterrows():
    index, hde_res, mre_res = dask_it(itertuple)
    break

In [ ]:
index

In [ ]:
mre_res

In [ ]:
hde_res

In [ ]:
import logging
import tempfile
import time
import numpy as np
from tqdm.auto import tqdm
from dask_jobqueue import SGECluster
from dask.distributed import Client, SSHCluster, LocalCluster, as_completed
from contextlib import nullcontext, ExitStack

# silence dask, configure this in ~/.config/dask/logging.yaml to be reliable
# https://docs.dask.org/en/latest/how-to/debug.html#logs
import logging

logging.basicConfig(level=logging.ERROR)
logging.getLogger("dask").setLevel(logging.WARNING)
logging.getLogger("distributed").setLevel(logging.WARNING)
logging.getLogger("distributed.worker").setLevel(logging.WARNING)

num_cores = 128
df_in_progress = None # for debugging, keep the global variable available

def main(dask_client, prepared_df):

    global df_in_progress, index, hde_res, mre_res
    df_in_progress = prepared_df.copy()
    df_in_progress["R_tot"] = np.nan
    df_in_progress["tau_R"] = np.nan
    df_in_progress["tau_single"] = np.nan
    df_in_progress["tau_double"] = np.nan
    df_in_progress["tau_R_details"] = None
    df_in_progress["tau_single_details"] = None
    df_in_progress["tau_double_details"] = None

    # dispatch, reading in parallel may be faster
    # create a list from the iterator, as dask map "no longer supports iterators" -.-
    futures = dask_client.map(dask_it, list(prepared_df.iterrows()))

    log.info("futures dispatched")

    # save once an hours
    last_save = time.time()
    for future in tqdm(as_completed(futures), total=len(futures), leave=True):

        index, hde_res, mre_res = future.result()

        # we use the .at[] method instead of .loc[] to be able to set
        # cells to dictionaries. (df.at can only access a single value at a time
        # df.loc can select multiple rows and/or columns)
        # https://stackoverflow.com/questions/13842088/set-value-for-particular-cell-in-pandas-dataframe-using-index
        df_in_progress.at[index, "R_tot"] = hde_res["R_tot"]
        df_in_progress.at[index, "tau_R"] = hde_res["tau_R"]
        df_in_progress.at[index, "tau_single"] = mre_res["tau_single"]
        df_in_progress.at[index, "tau_double"] = mre_res["tau_double"]

        df_in_progress.at[index, "tau_R_details"] = hde_res
        df_in_progress.at[index, "tau_single_details"] = mre_res["details_tau_single"]
        df_in_progress.at[index, "tau_double_details"] = mre_res["details_tau_double"]

        # save every hour or so
        if time.time() - last_save > 3600:
            time_str = time.strftime("%Y-%m-%d_%H-%M-%S")
            try:
                df_in_progress.reset_index().to_feather(f"{output_dir}/{output_name}_{time_str}.feather")
            except Exception as e:
                log.error(e)
            last_save = time.time()

    try:
        df_in_progress.reset_index().to_feather(f"{output_dir}/{output_name}_final.feather")
    except Exception as e:
        log.error(e)

    return df_in_progress


with ExitStack() as stack:
    # init dask using a context manager to ensure proper cleanup of remote compute
    dask_cluster = stack.enter_context(
        # rudabeh
        SGECluster(
            cores=32,
            memory="192GB",
            processes=32,
            job_extra_directives=["-pe mvapich2-sam 32"],
            log_directory="/scratch01.local/pspitzner/dask/logs",
            local_directory="/scratch01.local/pspitzner/dask/scratch",
            interface="ib0",
            walltime="24:00:00",
            worker_extra_args=[
                f'--preload \'import sys; sys.path.append("{extra_path}");\''
            ],
        )
        # local cluster
        # LocalCluster(local_directory=f"{tempfile.gettempdir()}/dask/")
    )
    dask_cluster.scale(cores=num_cores)
    dask_client = stack.enter_context(Client(dask_cluster))

    # for test use a smaller dataframe

    final_df = main(dask_client, meta_df)


2023-08-28 19:30:37,212 | INFO | notebook | main | futures dispatched


  0%|          | 0/12390 [00:00<?, ?it/s]

2023-08-28 20:30:39,405 | ERROR | notebook | main | ('Can only convert 1-dimensional array values', 'Conversion failed for column tau_single_details with type object')
2023-08-28 21:30:43,570 | ERROR | notebook | main | ('Can only convert 1-dimensional array values', 'Conversion failed for column tau_single_details with type object')


In [13]:
%watermark -v --iversions --packages mrestimator,hdestimator,scipy

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

mrestimator: 0.1.9b2
hdestimator: 0.10b2
scipy      : 1.11.1

re     : 2.2.1
pandas : 2.0.3
xarray : 2023.7.0
numpy  : 1.24.4
dask   : 2023.8.0
logging: 0.5.1.2
h5py   : 3.9.0
sys    : 3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:08:17) [GCC 12.2.0]



In [8]:
final_df2 = final_df.copy()
final_df.reset_index().to_hdf(f"{output_dir}/{output_name}_final.h5", key="meta_df")
loaded = pd.read_hdf(f"{output_dir}/{output_name}_final.h5", key="meta_df")
pd.testing.assert_frame_equal(final_df.reset_index(), loaded)
loaded

,unit_id,stimulus,session,block,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,spiketimes,R_tot,tau_R,tau_single,tau_double,tau_R_details,tau_single_details,tau_double_details
0,950987325,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.087891,1.999804,/data.nst/share/data/allen_visual_coding_neuro...,1796,"[0.029541016, 0.072021484, 0.38793945, 0.56079...",0.027605,0.306734,2.241440,2.241020,"{'firing_rate': 2.0738371954275348, 'firing_ra...","{'tau': 2.241440355939673, 'mre': 0.9977717777...","{'tau': 2.2410198071730076, 'mre': 0.997771360..."
1,950987344,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.868408,0.837720,/data.nst/share/data/allen_visual_coding_neuro...,753,"[5.1347656, 5.1501465, 12.0354, 12.419434, 12....",0.270567,0.012211,0.045573,0.043030,"{'firing_rate': 0.7508380960377098, 'firing_ra...","{'tau': 0.045573393213975646, 'mre': 0.8960911...","{'tau': 0.04303043931112166, 'mre': 0.89029998..."
2,950987340,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.287354,1.568388,/data.nst/share/data/allen_visual_coding_neuro...,1412,"[2.0043945, 2.2827148, 3.9108887, 4.941162, 4....",0.039402,0.059192,0.280984,0.245722,"{'firing_rate': 1.5801313991442945, 'firing_ra...","{'tau': 0.28098421748789454, 'mre': 0.98236279...","{'tau': 0.24572237330663918, 'mre': 0.97985745..."
3,950987352,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,894.181885,0.542395,/data.nst/share/data/allen_visual_coding_neuro...,485,"[0.14331055, 2.2165527, 3.510498, 9.070068, 9....",0.076247,0.125046,0.673576,0.606872,"{'firing_rate': 0.5503333173468898, 'firing_ra...","{'tau': 0.6735760723849045, 'mre': 0.992604416...","{'tau': 0.6068717836955805, 'mre': 0.991794874..."
4,950987362,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.613525,2.053045,/data.nst/share/data/allen_visual_coding_neuro...,1849,"[0.07299805, 0.1027832, 0.26831055, 0.49853516...",0.032636,0.115843,0.823834,0.742092,"{'firing_rate': 1.8513667968377936, 'firing_ra...","{'tau': 0.8238338588105363, 'mre': 0.993949195...","{'tau': 0.7420923475915008, 'mre': 0.993284941..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17944,951871993,natural_movie_three,746083955,3.0,VISrl,SUCCESS,600.296631,8.983893,/data.nst/share/data/allen_visual_coding_neuro...,5393,"[0.11743164, 0.1508789, 0.21679688, 0.29492188...",0.073165,0.007423,0.561941,0.215126,"{'firing_rate': 9.009579310922648, 'firing_rat...","{'tau': 0.561941385050045, 'mre': 0.9911417424...","{'tau': 0.21512599642991792, 'mre': 0.97702582..."
17945,951872268,natural_movie_three,746083955,3.0,VISrl,SUCCESS,597.750244,5.532411,/data.nst/share/data/allen_visual_coding_neuro...,3307,"[0.6315918, 0.69262695, 1.6867676, 1.7890625, ...",0.074235,0.075404,1.478688,0.016557,"{'firing_rate': 5.657953529937444, 'firing_rat...","{'tau': 1.4786882936742478, 'mre': 0.996624335...","{'tau': 0.01655729330665586, 'mre': 0.73935157..."
17946,951872311,natural_movie_three,746083955,3.0,VISrl,SUCCESS,598.611084,0.786821,/data.nst/share/data/allen_visual_coding_neuro...,471,"[2.0407715, 3.2368164, 3.4702148, 4.123291, 4....",0.073674,0.012557,0.398097,0.311044,"{'firing_rate': 0.7790150491543586, 'firing_ra...","{'tau': 0.39809727770807324, 'mre': 0.98751880...","{'tau': 0.3110436862624772, 'mre': 0.984053599..."
17947,951872945,natural_movie_three,746083955,3.0,VISrl,SUCCESS,594.244629,0.518305,/data.nst/share/data/allen_visual_coding_neuro...,308,"[3.5358887, 3.8791504, 5.395996, 7.1865234, 8....",0.045614,0.000000,3.802283,0.018774,"{'firing_rate': 0.5257108401952102, 'firing_ra...","{'tau': 3.8022828040875556, 'mre': 0.998685864...","{'tau': 0.018774130705340175, 'mre': 0.7661909..."


In [16]:
loaded_1 = pd.read_hdf(f"{output_dir}/meta_df_final.h5", "/meta_df")
loaded_2 = pd.read_hdf(f"{output_dir}/meta_df_2_final.h5", "/meta_df")

In [17]:
loaded_merged = pd.concat([loaded_1, loaded_2])
loaded_merged

,,,,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,R_tot,tau_R,tau_single,tau_double,tau_R_details,tau_single_details,tau_double_details
unit_id,stimulus,session,block,,,,,,,,,,,,,
950987325,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.087879,1.999804,/data.nst/share/data/allen_visual_coding_neuro...,1796,0.028897,0.292095,2.204573,2.204866,"{'firing_rate': 1.9997995746528745, 'firing_ra...","{'tau': 2.2045730546241966, 'mre': 0.997734557...","{'tau': 2.2048661904516664, 'mre': 0.997734858..."
950987344,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.868180,0.837720,/data.nst/share/data/allen_visual_coding_neuro...,753,0.254426,0.016040,0.049767,0.045802,"{'firing_rate': 0.8332684370376139, 'firing_ra...","{'tau': 0.04976678697174655, 'mre': 0.90441349...","{'tau': 0.04580227717841129, 'mre': 0.89658257..."
950987340,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.287381,1.568388,/data.nst/share/data/allen_visual_coding_neuro...,1412,0.039074,0.057306,0.263706,0.233516,"{'firing_rate': 1.5594974952515301, 'firing_ra...","{'tau': 0.26370610494040975, 'mre': 0.98121811...","{'tau': 0.23351615919178326, 'mre': 0.97881581..."
950987352,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,894.181973,0.542395,/data.nst/share/data/allen_visual_coding_neuro...,485,0.074347,0.126436,0.657249,0.594182,"{'firing_rate': 0.5412750157965074, 'firing_ra...","{'tau': 0.6572492090313748, 'mre': 0.992421399...","{'tau': 0.5941821956040216, 'mre': 0.991620379..."
950987362,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.613482,2.053045,/data.nst/share/data/allen_visual_coding_neuro...,1849,0.037868,0.110199,0.778819,0.749355,"{'firing_rate': 2.036386247175541, 'firing_rat...","{'tau': 0.7788194682871189, 'mre': 0.993600590...","{'tau': 0.7493554822530509, 'mre': 0.993349810..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951127614,natural_movie_one_more_repeats,839557629,merged_3.0_and_8.0,VISp,SUCCESS,818.458008,0.058647,/data.nst/share/data/allen_visual_coding_neuro...,48,0.011261,0.000000,5.241771,0.001162,"{'firing_rate': 0.05864672677956161, 'firing_r...","{'tau': 5.2417714824133785, 'mre': 0.999046578...","{'tau': 0.0011617555281603106, 'mre': 0.013516..."
951128175,natural_movie_one_more_repeats,839557629,merged_3.0_and_8.0,VISl,SUCCESS,913.291992,0.174095,/data.nst/share/data/allen_visual_coding_neuro...,159,0.094832,0.075662,0.136336,0.115506,"{'firing_rate': 0.1740948981435352, 'firing_ra...","{'tau': 0.13633607252766394, 'mre': 0.96399026...","{'tau': 0.11550580320940909, 'mre': 0.95763567..."
951185417,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,VISam,ERR_REC_LEN,957.861328,0.220282,/data.nst/share/data/allen_visual_coding_neuro...,211,0.159574,0.230245,1.579377,1.579400,"{'firing_rate': 0.22028156368590562, 'firing_r...","{'tau': 1.5793768186950516, 'mre': 0.996839200...","{'tau': 1.579400109924387, 'mre': 0.9968392468..."


In [18]:
utl.save_dataframe(loaded_merged, f"{output_dir}/meta_df_merged_final.h5")
reloaded = pd.read_hdf(f"{output_dir}/meta_df_merged_final.h5", "/meta_df")
reloaded

,,,,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,R_tot,tau_R,tau_single,tau_double,tau_R_details,tau_single_details,tau_double_details
unit_id,stimulus,session,block,,,,,,,,,,,,,
950987325,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.087879,1.999804,/data.nst/share/data/allen_visual_coding_neuro...,1796,0.028897,0.292095,2.204573,2.204866,"{'firing_rate': 1.9997995746528745, 'firing_ra...","{'tau': 2.2045730546241966, 'mre': 0.997734557...","{'tau': 2.2048661904516664, 'mre': 0.997734858..."
950987344,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.868180,0.837720,/data.nst/share/data/allen_visual_coding_neuro...,753,0.254426,0.016040,0.049767,0.045802,"{'firing_rate': 0.8332684370376139, 'firing_ra...","{'tau': 0.04976678697174655, 'mre': 0.90441349...","{'tau': 0.04580227717841129, 'mre': 0.89658257..."
950987340,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.287381,1.568388,/data.nst/share/data/allen_visual_coding_neuro...,1412,0.039074,0.057306,0.263706,0.233516,"{'firing_rate': 1.5594974952515301, 'firing_ra...","{'tau': 0.26370610494040975, 'mre': 0.98121811...","{'tau': 0.23351615919178326, 'mre': 0.97881581..."
950987352,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,894.181973,0.542395,/data.nst/share/data/allen_visual_coding_neuro...,485,0.074347,0.126436,0.657249,0.594182,"{'firing_rate': 0.5412750157965074, 'firing_ra...","{'tau': 0.6572492090313748, 'mre': 0.992421399...","{'tau': 0.5941821956040216, 'mre': 0.991620379..."
950987362,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.613482,2.053045,/data.nst/share/data/allen_visual_coding_neuro...,1849,0.037868,0.110199,0.778819,0.749355,"{'firing_rate': 2.036386247175541, 'firing_rat...","{'tau': 0.7788194682871189, 'mre': 0.993600590...","{'tau': 0.7493554822530509, 'mre': 0.993349810..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951127614,natural_movie_one_more_repeats,839557629,merged_3.0_and_8.0,VISp,SUCCESS,818.458008,0.058647,/data.nst/share/data/allen_visual_coding_neuro...,48,0.011261,0.000000,5.241771,0.001162,"{'firing_rate': 0.05864672677956161, 'firing_r...","{'tau': 5.2417714824133785, 'mre': 0.999046578...","{'tau': 0.0011617555281603106, 'mre': 0.013516..."
951128175,natural_movie_one_more_repeats,839557629,merged_3.0_and_8.0,VISl,SUCCESS,913.291992,0.174095,/data.nst/share/data/allen_visual_coding_neuro...,159,0.094832,0.075662,0.136336,0.115506,"{'firing_rate': 0.1740948981435352, 'firing_ra...","{'tau': 0.13633607252766394, 'mre': 0.96399026...","{'tau': 0.11550580320940909, 'mre': 0.95763567..."
951185417,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,VISam,ERR_REC_LEN,957.861328,0.220282,/data.nst/share/data/allen_visual_coding_neuro...,211,0.159574,0.230245,1.579377,1.579400,"{'firing_rate': 0.22028156368590562, 'firing_r...","{'tau': 1.5793768186950516, 'mre': 0.996839200...","{'tau': 1.579400109924387, 'mre': 0.9968392468..."
